<a href="https://colab.research.google.com/github/ElizavetaNosova/HSE_ML_homework/blob/master/EnsembleHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier,  BaggingClassifier, BaggingRegressor, RandomTreesEmbedding,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

In [4]:
data_url = 'https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv'
data = pd.read_csv(data_url, sep='\t')
data.head()

,COMPLAINT_ID,DATE,PRODUCT_ID,ISSUE_ID,cleaned_text
0,3178905,03/13/2019,44,318,go year . contact advis never took loan . advi...
1,3175952,03/12/2019,44,349,"mail valid debt xx/xx/19 valid receiv , receiv..."
2,3174747,03/09/2019,44,16,xx/xx/xxxx appli receiv onlin loan bluechip fi...
3,3173291,03/08/2019,44,16,xx/xx/xxxx appli receiv onlin loan . loan amou...
4,3172221,03/07/2019,44,48,told husband left bill . debt would pay within...


In [0]:
y = data["PRODUCT_ID"]
X = data["cleaned_text"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

 Voting Classifier (попробую другие параметры)

In [0]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [9]:
clf1 = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=1, C=1.5)
clf2 = RandomForestClassifier(n_estimators=75, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')

voting = Pipeline([
    ('vect', CountVectorizer(analyzer='word', max_features=500)),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('clf', eclf),
    ])
voting = voting.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
predictions = voting.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))
##Получилось увеличить метрики (в основном за счет параметров векторайзера)

Precision:   0.64
Recall:   0.64
F1-measure:   0.63
Accuracy:   0.64


Bagging

In [0]:
%matplotlib inline

import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
np.random.seed(0)

Одна модель, разные параметры

In [0]:
clf1 = KNeighborsClassifier(n_neighbors=50)
clf2 = KNeighborsClassifier(n_neighbors=15)    

bagging = BaggingClassifier(base_estimator=clf1, n_estimators=10, max_samples=0.8, max_features=0.8)

In [0]:
cv =  CountVectorizer( analyzer='word', max_features=1500, max_df=0.5)
XV = cv.fit_transform(X)

In [0]:
cv2= CountVectorizer(analyzer='word', max_features=500, max_df=0.5)
XV2 = cv2.fit_transform(X)

In [24]:
from statistics import mean
mean(cross_val_score(clf1, XV, y, cv=3, scoring='accuracy'))

0.4191856073730919

In [25]:
mean(cross_val_score(clf1, XV2, y, cv=3, scoring='accuracy'))

0.4444961791703877

In [26]:
mean(cross_val_score(clf2, XV, y, cv=3, scoring='accuracy'))

0.44075942393578793

In [27]:
mean(cross_val_score(clf2, XV2, y, cv=3, scoring='accuracy'))

0.45353515542427925

Для обеих моделей лучше векторизация XV2

In [29]:
mean(cross_val_score(bagging, XV2, y, cv=3, scoring='accuracy'))

0.4448578428231906

Не удалось превзойти результат на лучшей из моделей

Разные модели

In [0]:
clf1 = RandomForestClassifier(n_estimators=75, random_state=1)

In [40]:
mean(cross_val_score(clf1, XV2, y, cv=3, scoring='accuracy'))

0.6315512093155631

In [0]:
clf2 = KNeighborsClassifier(n_neighbors=15)

In [0]:
bagging = BaggingClassifier(base_estimator=clf1, n_estimators=10, max_samples=0.8, max_features=0.8)

In [43]:
mean(cross_val_score(bagging, XV2, y, cv=3, scoring='accuracy'))

0.6390234122516723

Превосходит лучшую из моделей-составляющих чуть меньше, чем на процент

Boosting

XGB

In [0]:
import xgboost as xgb

In [0]:
clf = xgb.XGBClassifier(objective='multi:softmax')
clf.fit(X_train,  y_train)
predictions = clf.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.68
Recall:   0.67
F1-measure:   0.67
Accuracy:   0.67


In [51]:
clf = xgb.XGBClassifier(objective='multi:sofmax',  n_estimators=500)
clf.fit(X_train,  y_train)
predictions = clf.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.72
Recall:   0.71
F1-measure:   0.72
Accuracy:   0.71


Stacking

In [0]:
from mlxtend.classifier import StackingClassifier

In [0]:
clf1 = KNeighborsClassifier(n_neighbors=15)
clf2 = RandomForestClassifier(random_state=1)
lr = LogisticRegression(max_iter=10000)
sclf = StackingClassifier(classifiers=[clf1, clf2], 
                          meta_classifier=lr)

In [66]:
clf_list = [clf1, clf2,  sclf]
labels = ['KNeighbors', 'RandomForest', 'Stacking']
clf_cv_mean = []
clf_cv_std = []
for clf, label in zip(clf_list, labels):
    scores = cross_val_score(clf, XV2, y, cv=3, scoring='accuracy')
    print ("Accuracy: %.2f (+/- %.2f) [%s]" %(scores.mean(), scores.std(), label))

Accuracy: 0.45 (+/- 0.01) [KNeighbors]
Accuracy: 0.64 (+/- 0.02) [RandomForest]
Accuracy: 0.64 (+/- 0.02) [Stacking]


Результат лучше, чем на отдельных составляющих

Лучший результат дал бустинг (с использованием xgboost)